# Integraging the Nordbord API to access team data
This guide is intended to provide instructions on how to interact with and use the [External Nordbord API.](https://prd-use-api-externalNordbord.valdperformance.com/swagger/index.html) This python class will provide the functionality to pull data from the Nordbord and Profiles API endpoints, to fetch and format data into csv(s) like you would on ValdHub. 

To use this class, you must first retrieve your ClientID, ClientSecret, and TenantID. In the vald_Nordbord Python file, replace the values of `self.client_id`, `self.client_secret`, and `self.tenant_id` with your retrieved values within the `Vald()` class.


### First, import required python packages.

In [1]:
import pandas as pd
import vald_nordbord
from vald_nordbord import Vald
from dotenv import load_dotenv
import os
import numpy as np
import time
import logging
import importlib
load_dotenv()
logging.basicConfig(level=logging.INFO)
importlib.reload(vald_nordbord) #used to ensure any changes in the vald_Nordbord python file are reflected in this notebook

<module 'vald_nordbord' from 'C:\\Users\\Catapult\\Documents\\Docker\\Projects\\vald\\vald_nordbord\\vald_nordbord.py'>

### Our class is called vald, let's create an instance of it and inspect it's attributes and methods
The attributes defined in this class are

| Attribute                 | Description                                                                                             |
|----------------------------|---------------------------------------------------------------------------------------------------------|
| `client_id`                | The unique identifier for the client application, used to authenticate Vald API requests.                    |
| `client_secret`            | A secret key associated with the `client_id`, used to securely authenticate Vald API requests.                |
| `tenant_id`                | The identifier for the specific tenant or organization within the Vald API system.                          |
| `Nordbord_api_url`       | The URL endpoint for accessing the [External Nordbord API](https://prd-use-api-externalNordbord.valdperformance.com/swagger/index.html), used to retrieve performance metrics and data.        |
| `groupnames_api_url`       | The URL endpoint for accessing the [External Tenants API](https://prd-use-api-externaltenants.valdperformance.com/swagger/index.html) to retrieve group (team) names related to the athletes or tests.        |
| `profiles_api_url`         | The URL endpoint for accessing the [External Profiles API](https://prd-use-api-externalprofile.valdperformance.com/swagger/index.html) to retrieve athlete profile information.                         |
| `vald_master_file_path`    | The file path to the master file containing all Nordbord data.                                        |
| `base_directory`           | The base directory on the local system where files and data related to the Vald system are stored.       |

`client_id`, `client_secret`, and `tenant_id` will be stored in a `.env` file for security purposes. You can retrieve these credentials from ValdHub or by reaching out to your Vald support representative. These values remain consistent across your organization, meaning you'll use the same credentials to interact with all of Vald's external APIs.


### Now let's look at the primary methods of this class

| Method                  | Description                                                                                                               |
|-------------------------|---------------------------------------------------------------------------------------------------------------------------|
| `get_last_update`        | Retrieves the last test date from the MasterFile and adds a 1-millisecond increment to ensure uniqueness for API requests. |
| `sanitize_filename`      | Replaces any special characters in a filename with underscores for safe file saving.                                       |
| `sanitize_foldername`    | Replaces any special characters in a folder name with underscores or spaces to ensure safe folder creation.                |
| `get_access_token`       | Requests an access token using `client_id` and `client_secret` to authenticate API requests.                               |
| `fetch_data`             | Fetches data from a given API URL using provided headers, returning JSON data if the response is successful.               |
| `get_tests`              | Retrieves test data from the [Nordbord API](https://prd-use-api-extNordbord.valdperformance.com/index.html), retrieves athlete information from the Profiles API, <br> retrieves group (team) information from the Groupnames API, and combines group and athlete data in parallel.  |
| `modify_df`              | Modifies and reformats a DataFrame by adding UTC dates/times and renaming key columns for clarity.                         |
| `update_Nordbord`      | Updates the master data file with the latest test data from the Nordbord system.                                         |
| `update_master_file`     | Appends new data to the existing master file or creates a new file if it does not exist.                                    |
| `save_dataframes`        | Saves team-specific test data in individual files, updating existing files if necessary.                                    |
| `save_master_file`       | Saves the master DataFrame to the specified file, creating necessary directories if they don't exist.                      |
| `data_to_groups`         | Organizes the retrieved test data into teams/groups and separates it by test type.                                         |
| `get_data_until_today`   | Fetches test data from the API until today’s date and saves it after processing and filtering for duplicates.              |
| `populate_folders`       | Sets up the folder structure and updates the data by calling the relevant methods, then saves the team/group data.         |


### `get_tests`

The `get_tests` method accepts two parameters: `start_date` and `pageno`. 

- The **`start_date`** parameter specifies the date to be plugged into the `TestFromUtc` parameter of the `/tests` API input. This allows the function to retrieve data starting from the specified date.
- The **`pageno`** parameter indicates which page of data to fetch from the endpoint.

This structure provides an intuitive way to interact with the API, as it retrieves tests starting from a certain date and paginates through the results. However, you could modify the function to include additional parameters like:

- **`TestToUtc`**: Specifies the end date for filtering tests.
- **`ModifiedFromUtc`**: Filters tests based on the date they were last modified.
- **`GroupUnderTestId`**: Filters tests by specific group IDs.

These parameters can enhance flexibility depending on your specific implementation needs.


#### Functionality

1. **Access Token Retrieval**:
   - The method starts by attempting to get the access token using the `get_access_token` function. If it fails to retrieve the token, it prints an error message and exits the function.

2. **API URL Construction**:
   - An API URL, `api_url` is constructed using the provided `start_date` and `pageno`, formatted as a query string.

3. **Fetching Tests Data**:
   - The method calls the `fetch_data` function with the constructed API URL and authorization headers to retrieve the tests data. If the response is empty (i.e., `None`), it returns an empty DataFrame.

4. **Group Names Retrieval**:
   - The method then constructs a second API URL to fetch group names associated with the tenant. It creates a mapping of group IDs to group names for later use.

5. **Concurrent Data Fetching for Profiles**:
   - Using a `ThreadPoolExecutor`, it concurrently fetches profile data for each test in the `tests_data` using the `fetch_data` method. This is done to improve efficiency by making multiple requests in parallel.
   - As each profile data is retrieved, it adds the `Name` (composed of given and family names) and the associated `Groups` (derived from group IDs) to each test record.

6. **Flattening Nested JSON**:
   - The method includes a nested function, `flatten_json`, which is responsible for transforming nested JSON structures into a flat dictionary format. This is useful for converting complex data into a more manageable format.
   - The method calls this function on each record in the `tests_data` to create a list of flattened records.

7. **DataFrame Creation**:
   - Finally, it converts the flattened data into a pandas DataFrame and prints a completion message before returning the DataFrame.

In [2]:
Nordbord = Vald()
start_date = '2024-10-01T00:00:00Z'
#Store the data in october_data
october_data = Nordbord.get_tests(start_date, 1)
#If you are seeing "Failed to retrieve access token", ensure you have properly set up your .env file.

Getting tests starting from 2024-10-01T00:00:00Z on page number 1
Data retrieval complete.(NB)


### JSON Data Retrieved from the Nordbord API

The following JSON structure is an example value of the schema of the /tests endpoint:

```json
{
  "tests": [
    {
      "athleteId": "3fa85f64-5717-4562-b3fc-2c963f66afa6",
      "testId": "3fa85f64-5717-4562-b3fc-2c963f66afa6",
      "modifiedUtc": "2024-10-11T20:28:43.341Z",
      "testDateUtc": "2024-10-11T20:28:43.341Z",
      "testTypeId": "3fa85f64-5717-4562-b3fc-2c963f66afa6",
      "testTypeName": "string",
      "notes": "string",
      "device": "string",
      "leftAvgForce": 0,
      "leftImpulse": 0,
      "leftMaxForce": 0,
      "leftTorque": 0,
      "leftCalibration": 0,
      "rightAvgForce": 0,
      "rightImpulse": 0,
      "rightMaxForce": 0,
      "rightTorque": 0,
      "rightCalibration": 0,
      "leftRepetitions": 0,
      "rightRepetitions": 0
    }
  ],
  "page": 0,
  "pageCount": 0
}
```
The `get_tests` method attaches the athlete's name and group (team) to each row of data by matching the `athleteId` with the corresponding athlete profile. This is achieved by retrieving the athlete's name from the profiles API, and the athlete's group (team) from the tenants API. The resulting dataframe looks like this:

In [3]:
pd.set_option('display.max_columns', None)
#Remove this line. Names de-identified for privacy reasons
october_data[['Name', 'testId', 'athleteId']] = np.nan
october_data.head()

,athleteId,testId,modifiedUtc,testDateUtc,testTypeId,testTypeName,notes,device,leftAvgForce,leftImpulse,leftMaxForce,leftTorque,leftCalibration,rightAvgForce,rightImpulse,rightMaxForce,rightTorque,rightCalibration,leftRepetitions,rightRepetitions,Name,Groups
0,NaN,NaN,2024-10-11T16:45:26.014Z,2024-10-11T16:45:24.203Z,596a6004-553f-4afa-aef0-1474903b462e,Nordic,,NordBord-1891,201.416667,4161.320,223.75,0.000000,0.0,201.166667,4095.280,220.00,0.000000,0.0,3,3,NaN,WVB
1,NaN,NaN,2024-10-11T16:45:23.083Z,2024-10-11T16:45:21.370Z,596a6004-553f-4afa-aef0-1474903b462e,Nordic,,NordBord-1901,137.500000,1135.325,137.50,63.731250,0.0,186.500000,2944.255,186.50,86.442750,0.0,0,0,NaN,WVB
2,NaN,NaN,2024-10-11T16:16:57.135Z,2024-10-11T16:16:56.972Z,1ccf7d5f-307d-4d1f-9fb3-1d8a9878d94a,ISO 30,,NordBord-1871,429.250000,2570.565,505.50,211.551750,0.0,289.750000,1868.935,323.50,135.384750,0.0,3,3,NaN,MSOC
3,NaN,NaN,2024-10-08T17:06:20.022Z,2024-10-08T17:06:19.786Z,1ccf7d5f-307d-4d1f-9fb3-1d8a9878d94a,ISO 30,,NordBord-1891,257.083333,1841.555,277.75,91.240875,0.0,244.583333,1896.660,254.75,83.685375,0.0,3,3,NaN,WSOC
4,NaN,NaN,2024-10-08T17:04:09.371Z,2024-10-08T17:04:09.236Z,1ccf7d5f-307d-4d1f-9fb3-1d8a9878d94a,ISO 30,,NordBord-1901,310.875000,1692.900,316.25,118.119375,0.0,316.500000,1664.915,331.00,123.628500,0.0,2,2,NaN,WSOC


The resulting DataFrame contains all the information present in the JSON structure. The `modify_df` function will add Imbalances to your data, and rename the columns to improve readability. Feel free to modify the `modify_df` function to fit your specific needs.

In [4]:
october_data_cleaned = Nordbord.modify_df(october_data)
october_data_cleaned.head(5)

,athleteId,testId,modifiedUtc,testDateUtc,testTypeId,Athlete,Groups,ExternalId,Date,Time UTC,Device,Test,L Reps,R Reps,L Max Force (N),R Max Force (N),Max Imbalance (%),L Max Torque (Nm),R Max Torque (Nm),L Avg Force (N),R Avg Force (N),Avg Imbalance (%),L Max Impulse (Ns),R Max Impulse (Ns),Impulse Imbalance (%),rightCalibration,leftCalibration,Notes
0,NaN,NaN,2024-10-11T16:45:26.014Z,2024-10-11T16:45:24.203Z,596a6004-553f-4afa-aef0-1474903b462e,NaN,WVB,,10/11/2024,04:45 PM,NordBord-1891,Nordic,3,3,223.75,220.00,-1.675978,0.000000,0.000000,201.416667,201.166667,-0.124121,4161.320,4095.280,-1.586996,0.0,0.0,
1,NaN,NaN,2024-10-11T16:45:23.083Z,2024-10-11T16:45:21.370Z,596a6004-553f-4afa-aef0-1474903b462e,NaN,WVB,,10/11/2024,04:45 PM,NordBord-1901,Nordic,0,0,137.50,186.50,26.273458,63.731250,86.442750,137.500000,186.500000,26.273458,1135.325,2944.255,61.439311,0.0,0.0,
2,NaN,NaN,2024-10-11T16:16:57.135Z,2024-10-11T16:16:56.972Z,1ccf7d5f-307d-4d1f-9fb3-1d8a9878d94a,NaN,MSOC,,10/11/2024,04:16 PM,NordBord-1871,ISO 30,3,3,505.50,323.50,-36.003956,211.551750,135.384750,429.250000,289.750000,-32.498544,2570.565,1868.935,-27.294778,0.0,0.0,
3,NaN,NaN,2024-10-08T17:06:20.022Z,2024-10-08T17:06:19.786Z,1ccf7d5f-307d-4d1f-9fb3-1d8a9878d94a,NaN,WSOC,,10/08/2024,05:06 PM,NordBord-1891,ISO 30,3,3,277.75,254.75,-8.280828,91.240875,83.685375,257.083333,244.583333,-4.862237,1841.555,1896.660,2.905370,0.0,0.0,
4,NaN,NaN,2024-10-08T17:04:09.371Z,2024-10-08T17:04:09.236Z,1ccf7d5f-307d-4d1f-9fb3-1d8a9878d94a,NaN,WSOC,,10/08/2024,05:04 PM,NordBord-1901,ISO 30,2,2,316.25,331.00,4.456193,118.119375,123.628500,310.875000,316.500000,1.777251,1692.900,1664.915,-1.653081,0.0,0.0,


### Saving the Data to .csv Using `save_masterfile`, `data_to_groups` and `save_dataframes`

### `save_masterfile`
- The `save_masterfile` function consolidates all sports data into a single master CSV file.
- It checks if the master file already exists; if it does, new data is appended to ensure no information is lost.
- This master file provides a comprehensive overview of all recorded metrics, making it easier for analysts to access a unified dataset for broader analysis.

### `data_to_groups` 
- The function organizes a given DataFrame into a nested dictionary (`teams_data`) based on unique groups found in the `Groups` column, allowing for structured data management.
- For each group, it further categorizes the data by unique test names found in the `testName` column, storing the corresponding test data in the nested dictionary structure.
- The result is a comprehensive dictionary where each key represents a group, and each value is another dictionary containing test names as keys and their associated test data as values, facilitating easy access to specific datasets.


### `save_dataframes`
- The `save_dataframes` function organizes the data into distinct folders based on each sport, enhancing data accessibility.
- Inside each sport's folder, it creates separate CSV files for each test type, allowing for targeted analysis by strength performance coaches.
- This structured organization facilitates efficient data retrieval and analysis, enabling coaches to quickly find and utilize the information specific to their needs.

You can customize these functions, as well as the `vald_master_file_path` and `base_directory`, to better suit your requirements. One suggestion is to set the base directory to your organization's shared OneDrive. This way, the DataFrames will be saved in the cloud, making them accessible to whoever may need to access the data.

### Execute the following code block to save the `october_data_cleaned` DataFrame to your local machine as multiple .csv files, organized by sport. This will help you visualize the directory structure.

In [5]:
Nordbord.save_master_file(october_data_cleaned)
october_data_cleaned_by_sport = Nordbord.data_to_groups(october_data_cleaned)
Nordbord.save_dataframes(october_data_cleaned_by_sport)

Directory to be created: data\master_files(NB)
Saved master file data\master_files\nordbord_allsports.csv(NB)
No existing file found for WVB - Nordic. Creating a new one.
Created and saved new file data\wvb\NordBord\wvb_nordic.csv
No existing file found for MSOC - ISO 30. Creating a new one.
Created and saved new file data\msoc\NordBord\msoc_iso_30.csv
No existing file found for WSOC - ISO 30. Creating a new one.
Created and saved new file data\wsoc\NordBord\wsoc_iso_30.csv
No existing file found for SB - ISO 30. Creating a new one.
Created and saved new file data\sb\NordBord\sb_iso_30.csv


The previous example was simplified to focus on illustrating how each function operates independently. However, it is important to note that the `october_data_cleaned` DataFrame only contains a single page of data, comprising 50 records, which is the limit imposed by the API's `/tests` endpoint. To retrieve more than one page of data, we will need to utilize the `get_data_until_today` function.


### `get_data_until_today`
- The function retrieves test data from a specified `start_date` until today, fetching data in batches, where each batch is one 50 records "page", and aggregating it into a single DataFrame (`new_data`).
- It checks for existing data in the master file, and if any duplicates are found based on the 'id' column, these duplicates are removed from the new data.
- After formatting the new data with the `modify_df` function, it saves this updated DataFrame to the master file and then processes the data into groups using the `data_to_groups` and `save_dataframes` functions, ultimately saving the organized team data to appropriate files.

Execute the following code block to retrieve and save DataFrames from the specified `start_date` to the most recent available record. You can adjust the `start_date` as needed. Keep in mind that if your `start_date` is set to a date earlier than the data in `october_data_cleaned`, any duplicate records will be removed from your master and group data files.

In [6]:
start_date = '2024-10-01T00:00:00Z'
Nordbord.get_data_until_today(start_date)
#Name and id will be printed as NaN until you remove line 268 from vald_Nordbord.py

Getting tests starting from 2024-10-01T00:00:00Z on page number 1
Data retrieval complete.(NB)
Getting tests starting from 2024-10-01T00:00:00Z on page number 2
Data retrieval complete.(NB)
Getting tests starting from 2024-10-01T00:00:00Z on page number 3
No new tests were found, the master file is up to date.
Saved master file data\master_files\nordbord_allsports.csv(NB)
No existing file found for WVB - Nordic. Creating a new one.
Appended data to data\wvb\NordBord\wvb_nordic.csv
No existing file found for MSOC - ISO 30. Creating a new one.
Appended data to data\msoc\NordBord\msoc_iso_30.csv
No existing file found for WSOC - ISO 30. Creating a new one.
Appended data to data\wsoc\NordBord\wsoc_iso_30.csv
No existing file found for SB - ISO 30. Creating a new one.
Appended data to data\sb\NordBord\sb_iso_30.csv
No existing file found for MVB - Nordic. Creating a new one.
Created and saved new file data\mvb\NordBord\mvb_nordic.csv
No existing file found for MTEN - ISO 30. Creating a new 

Finally, the `update_Nordbord` function will identify the most recent date recorded in your master file. It will then call `get_data_until_today`, using this latest date as the parameter, effectively retrieving any new tests that are not already present in your .csv files.


In [7]:
Nordbord.update_nordbord

<bound method Vald.update_nordbord of <vald_nordbord.Vald object at 0x0000023E719843B0>>